Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [5]:
# from DT_atualiza_settings import *
# from DT_StatusInvest import SI
# SI(mercado = 'Acoes' )

In [6]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Codigos-em-financas/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [7]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [8]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
GOAU4,8.93,29.44,83.922233,58.27,1.55,9.029604,45.08,404.82,5.78,0.70
GOAU3,9.21,29.44,0.489116,58.27,1.55,9.029604,45.08,389.47,5.96,0.70
EUCA4,13.35,68.52,0.382901,27.91,4.25,1.323676,51.66,286.97,3.14,1.43
JHSF3,3.78,28.32,9.872121,7.73,0.93,2.575400,12.72,236.51,4.06,2.43
LOGG3,18.86,30.65,7.541213,41.04,3.91,1.657024,60.09,218.61,4.83,3.77
VBBR3,18.20,23.45,234.516184,18.25,8.18,20.365800,57.96,218.46,2.22,0.73
MTRE3,3.10,23.11,1.633493,9.81,0.42,0.327902,9.63,210.65,7.45,5.09
SOMA3,5.95,80.87,90.651390,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
SCAR3,17.65,73.93,0.404809,28.93,4.32,1.019064,53.03,200.45,4.09,2.86
BRAP3,15.71,8.66,0.768544,21.50,4.59,6.548004,47.12,199.94,3.42,-0.17


outros criterios adicionais

P/L > 0 e DIVIDA LIQUIDA / EBIT < 3

In [9]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
GOAU4,8.93,29.44,83.922233,58.27,1.55,9.029604,45.08,404.82,5.78,0.70
GOAU3,9.21,29.44,0.489116,58.27,1.55,9.029604,45.08,389.47,5.96,0.70
EUCA4,13.35,68.52,0.382901,27.91,4.25,1.323676,51.66,286.97,3.14,1.43
JHSF3,3.78,28.32,9.872121,7.73,0.93,2.575400,12.72,236.51,4.06,2.43
VBBR3,18.20,23.45,234.516184,18.25,8.18,20.365800,57.96,218.46,2.22,0.73
SOMA3,5.95,80.87,90.651390,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
SCAR3,17.65,73.93,0.404809,28.93,4.32,1.019064,53.03,200.45,4.09,2.86
BRAP3,15.71,8.66,0.768544,21.50,4.59,6.548004,47.12,199.94,3.42,-0.17
BRAP4,17.17,8.66,46.671216,21.50,4.59,6.548004,47.12,174.43,3.74,-0.17
DMVF3,5.94,552.01,0.424319,17.65,0.62,0.300581,15.69,164.14,9.51,-0.26


remove empresas repetidas, mantem primeira ocorrencia

In [10]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
GOAU4,8.93,29.44,83.922233,58.27,1.55,9.029604,45.08,404.82,5.78,0.70
EUCA4,13.35,68.52,0.382901,27.91,4.25,1.323676,51.66,286.97,3.14,1.43
JHSF3,3.78,28.32,9.872121,7.73,0.93,2.575400,12.72,236.51,4.06,2.43
LOGG3,18.86,30.65,7.541213,41.04,3.91,1.657024,60.09,218.61,4.83,3.77
VBBR3,18.20,23.45,234.516184,18.25,8.18,20.365800,57.96,218.46,2.22,0.73
MTRE3,3.10,23.11,1.633493,9.81,0.42,0.327902,9.63,210.65,7.45,5.09
SOMA3,5.95,80.87,90.651390,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
SCAR3,17.65,73.93,0.404809,28.93,4.32,1.019064,53.03,200.45,4.09,2.86
BRAP3,15.71,8.66,0.768544,21.50,4.59,6.548004,47.12,199.94,3.42,-0.17
DMVF3,5.94,552.01,0.424319,17.65,0.62,0.300581,15.69,164.14,9.51,-0.26


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

# Stocks

In [17]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [18]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,134.88,0.71,29.95,6.55,3.26,54.30,24.84,19.80,23.88,25.15,1.27,0.35,5.93,18.70,-4.90,2.09,21.85,10.88,14.91,0.50,0.50,0.55,4.75,3.77,NaN,20.65,4.51,7.58,3.860677e+10
1,AA,34.75,1.15,149.63,1.74,0.64,15.56,3.74,0.50,20.17,25.69,5.52,0.48,0.75,5.91,-0.98,1.45,1.16,0.43,2.05,0.37,0.63,0.85,2.66,NaN,NaN,19.96,0.23,-1.37,8.977829e+09
2,AACG,1.07,NaN,-3.25,1.47,0.47,51.68,-31.49,-32.34,-3.34,-2.73,0.61,-0.27,1.05,-1.01,-0.54,0.23,-45.26,-14.39,-39.89,0.32,0.68,0.44,176.03,NaN,NaN,0.73,-0.33,-0.08,3.367707e+07
3,AACQ,9.99,NaN,-93.43,180.94,1.25,NaN,NaN,NaN,-27.75,-27.74,0.01,-0.07,NaN,2094.26,-1.25,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
4,AACQU,11.05,NaN,-103.35,200.14,1.38,NaN,NaN,NaN,-30.69,-27.74,0.01,-0.07,NaN,2316.47,-1.38,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5240,ZUO,10.02,NaN,-20.82,8.36,1.80,67.42,-10.54,-16.34,-32.28,-28.30,3.99,-1.03,3.40,3.65,-8.80,2.62,-40.16,-8.63,-8.89,0.21,0.79,0.53,12.93,NaN,NaN,1.20,-0.48,0.40,1.540242e+09
5241,ZVO,0.09,NaN,-0.08,0.47,0.04,35.09,-17.75,-17.80,-0.08,0.59,0.66,-4.11,0.01,-1.38,-0.07,0.95,-619.73,-47.91,-619.73,0.08,0.92,2.69,-12.86,NaN,NaN,0.19,-1.20,0.00,3.078549e+06
5242,ZYME,13.33,NaN,-8.00,2.48,1.87,100.00,-189.75,-182.75,-7.71,-5.19,2.52,-0.81,14.63,3.50,-6.35,4.10,-30.97,-23.33,-33.35,0.75,0.25,0.13,7.47,NaN,NaN,5.33,-1.65,0.05,9.098718e+08
5243,ZYNE,1.30,NaN,-1.87,2.42,1.73,NaN,NaN,NaN,-1.87,-0.91,0.96,-1.24,NaN,2.52,-59.32,3.43,-129.50,-92.85,-129.50,0.72,0.28,0.00,NaN,NaN,NaN,0.54,-0.70,4.43,7.012122e+07


In [19]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [20]:
fundsSI =  funds[ #(funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
b_print(fundsSI,100)

TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
AAON,101.00,0.32,43.00,10.30,7.97,35.65,20.18,15.78,33.64,33.84,0.20,0.06,6.79,28.50,-13.62,3.06,23.96,18.52,22.70,0.77,0.23,1.17,21.91,33.22,NaN,9.80,2.35,3.40,8.209240e+09
AAPL,245.89,0.41,38.45,55.38,10.74,46.52,31.77,24.30,29.40,29.74,0.34,0.64,9.34,-332.30,-17.53,0.92,144.03,27.94,58.79,0.19,0.81,1.15,8.49,11.15,NaN,4.44,6.40,0.56,3.696782e+12
AAWW,102.48,NaN,8.25,0.96,0.44,53.54,11.66,7.82,5.53,8.47,2.94,0.51,0.65,-63.29,-0.53,0.96,11.61,5.31,7.87,0.46,0.54,0.68,16.10,9.75,NaN,107.04,12.43,-0.30,2.934330e+09
ABB,37.12,NaN,17.32,5.68,1.80,32.37,17.56,13.65,13.46,13.66,0.20,0.09,2.36,89.85,-3.35,1.04,32.76,10.36,24.67,0.32,0.68,0.76,3.17,2.26,NaN,6.54,2.14,0.15,6.900207e+10
ABEV,1.93,2.16,10.74,1.72,1.11,49.32,23.08,17.72,8.24,7.33,-0.91,-0.19,1.90,24.00,-1.57,1.19,16.06,10.35,19.01,0.64,0.36,0.58,4.84,1.02,NaN,1.11,0.18,2.77,3.012242e+10
ABMD,381.02,NaN,64.40,11.16,10.13,81.32,31.76,24.84,50.36,47.95,-2.40,-0.53,16.00,18.50,-27.11,7.93,17.32,15.72,17.32,0.91,0.09,0.63,18.30,21.24,NaN,34.15,5.92,0.82,1.718065e+10
ABT,135.80,1.22,17.57,4.94,2.89,55.41,18.05,31.95,31.11,31.92,0.81,0.13,5.61,24.80,-4.08,1.67,28.12,16.46,1.91,0.59,0.41,0.52,5.63,29.45,NaN,27.48,7.73,0.13,2.355391e+11
ABTX,41.63,NaN,10.98,1.18,0.12,0.00,38.18,28.90,8.31,-11.75,-20.06,-2.84,3.17,NaN,-0.12,NaN,10.73,1.12,11.72,0.10,0.90,0.04,19.36,28.98,NaN,35.34,3.79,3.01,8.309431e+08
ACA,93.49,0.16,35.49,1.87,1.05,19.84,8.00,5.17,22.94,25.36,2.41,0.20,1.83,4.05,-1.63,3.61,5.28,2.95,4.66,0.56,0.44,0.57,9.58,16.03,NaN,49.90,2.63,-0.64,4.560631e+09


In [21]:
fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
# fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
AAWW,102.48,9.75,107.04,12.43,2.934330,173.02,68.83,8.25,2.94
ABTX,41.63,28.98,35.34,3.79,0.830943,54.90,31.88,10.98,-20.06
ABEV,1.93,1.02,1.11,0.18,30.122419,2.12,9.84,10.74,-0.91
ACA,93.49,16.03,49.90,2.63,4.560631,54.34,-41.88,35.49,2.41
ABT,135.80,29.45,27.48,7.73,235.539111,69.13,-49.09,17.57,0.81
ABB,37.12,2.26,6.54,2.14,69.002075,17.75,-52.18,17.32,0.20
AAON,101.00,33.22,9.80,2.35,8.209240,22.76,-77.47,43.00,0.20
ABMD,381.02,21.24,34.15,5.92,17.180649,67.44,-82.30,64.40,-2.40
AAPL,245.89,11.15,4.44,6.40,3696.781945,25.29,-89.71,38.45,0.34
